<a href="https://colab.research.google.com/github/wgova/time_series_trade/blob/master/kmeans_basecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tslearn
import pandas as pd
import numpy as np
import glob
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 747kB 2.8MB/s 


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
import os
os.chdir('/content/drive/My Drive/phd_data/sitc/')
os.getcwd()

'/content/drive/My Drive/phd_data/sitc'

In [0]:
def read_filedata(filenames):
    dfs = []
    for filename in filenames:
        original_df = pd.read_csv(filename, index_col=None, header=0)
        dfs.append(original_df)
    return dfs

def time_series_data(dataframes, target_col=None):
    tsdata = []
    for i, df in enumerate(dataframes):
        tsdata.append(df[target_col].values.tolist()[:])
        len_max = 0
        for ts in tsdata:
            if len(ts) > len_max:
                len_max = len(ts)
        for i, ts in enumerate(tsdata):
            len_add = len_max - len(ts)
            tsdata[i] = ts + [ts[-1]] * len_add
    
    tsdata = np.array(tsdata)
    return tsdata

def transform_vector(time_series_array):
    stack_list = []
    for j in range(len(time_series_array)):
        data = np.array(time_series_array[j])
        data = data.reshape((1, len(data))).T
        stack_list.append(data)
    stack_data = np.stack(stack_list, axis=0)
    return stack_data

In [0]:
list_df = []
def create_df():
  exclude_list = ['export_rca','export_val_growth_pct','export_val_growth_pct_5', 'export_val_growth_val','export_val_growth_val_5', 
                'import_rca', 'import_val','import_val_growth_pct', 'import_val_growth_pct_5','import_val_growth_val', 'import_val_growth_val_5','sitc_id_len']
  trade_data = [i for i in os.listdir(os.getcwd()) if i.endswith('.csv') and 'sitc' in i]
  for file in trade_data:
    trade_history = []
    file_name = pd.read_csv(file).drop(exclude_list, axis=1)
    trade_history.append(file_name)
  products_dataframe = pd.concat(trade_history)
  return products_dataframe

xtrade_years = create_df()
xtrade_years['year'] = pd.to_datetime(trade_dframe['year'], format='%Y')

In [71]:
countries = pd.read_csv('list_countries.csv',usecols=['id','name'],
                        na_values='',na_filter=True)
countries.rename(columns=lambda x: x.replace('name','country'), inplace=True)
countries.rename(columns=lambda x: x.replace('id','origin_id'), inplace=True)
trade_dframe = xtrade_years.merge(countries, on=['origin_id'],how='left')
trade_dframe.head()

,export_val,origin_id,sitc_id,year,country
0,18000.0,afdza,107188,1962-01-01,Algeria
1,65000.0,afmar,107188,1962-01-01,Morocco
2,1000.0,afuga,107188,1962-01-01,Uganda
3,5000.0,afzaf,107188,1962-01-01,South Africa
4,2000.0,ashkg,107188,1962-01-01,Hong Kong


In [0]:
products = pd.read_csv('list_products.csv',usecols=['id','name'],)
# pd.to_numeric(products['id'],errors='coerce')
products.apply(lambda col:pd.to_numeric('id', errors='coerce'))
# products['name'].astype()
products.rename(columns={"id":"sitc_id","name":"product"},inplace=True)
trade_dframe = trade_dframe.merge(products, on=['sitc_id'],how='left')
products.info()

In [47]:
# filenames = sorted(glob.glob('sitc-*.csv'))
# trade_dframe['export_val'].replace('',0, inplace=True)
billion = 1000000000
trade_dframe['export_val'] = trade_dframe['export_val']/billion
df = trade_dframe[trade_dframe['export_val'].notna()]
# df=df.set_index(['sitc_id',"country","year"])
# df.drop(['origin_id'])
# dfa = df.squeeze()
df.isnull().sum()
df.head()

export_val origin_id
sitc_id country      year                               
107188  Algeria      1962-01-01  1.800000e-113     afdza
        Morocco      1962-01-01  6.500000e-113     afmar
        Uganda       1962-01-01  1.000000e-114     afuga
        South Africa 1962-01-01  5.000000e-114     afzaf
        Hong Kong    1962-01-01  2.000000e-114     ashkg

In [46]:
#tsdata = time_series_data(dataframes=dfa, target_col='export_rca')
stack_data = transform_vector(time_series_array=dfa)

TypeError: ignored

In [7]:
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

X, _ = make_blobs(centers=)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
# tsdata = time_series_data(dataframes=df, target_col='export_rca')
stack_data = transform_vector(time_series_array=df)

KeyError: ignored

In [0]:
seed = 0
np.random.seed(seed)
#相互相関を計算するために、正規化する必要があります。
#TimeSeriesScalerMeanVarianceがデータを正規化してくれるクラスになります。
stack_data = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0).fit_transform(stack_data)

#KShapeクラスのインスタンス化
ks = KShape(n_clusters=2, n_init=10, verbose=True, random_state=seed)
y_pred = ks.fit_predict(stack_data)

#クラスタリングして可視化
plt.figure(figsize=(16,9))
for yi in range(2):
    plt.subplot(2, 1, 1 + yi)
    for xx in stack_data[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    #plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.title("Cluster %d" % (yi + 1))

plt.tight_layout()
plt.show()

ValueError: ignored

Reference: https://github.com/masatakashiwagi/Analysis_tslearn